In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
TABLE = 'Tuition'

In [2]:
url = 'https://www.mcdaniel.edu/admissions-cost/cost-financial-aid/graduate-tuition-fees'
page = requests.get(url)

with open(TABLE + '.html', 'w', encoding='utf-8') as file:
    file.write(page.text)

In [3]:
with open(TABLE + '.html', 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')

In [4]:
degree = 'M.S. in Data Analytics'
button = soup.find('h3', string=degree)
div = button.find_next('div')
p_tags = div.find_all('p')

cols = ['Program', 'Tuition (Per Credit Hour)', 'Residency Fee', 'Practicum Fee']

tuition_fee = p_tags[0].text
residency_fee = p_tags[1].text.split(' - ')[-1]
practicum_fee = p_tags[2].text.split(' - ')[-1]

data_1 = {'Program': degree, 'Tuition (Per Credit Hour)': tuition_fee, 
        'Residency Fee': residency_fee, 'Practicum Fee': practicum_fee}


df1 = pd.DataFrame([data_1], columns=cols)

In [5]:
ul = soup.find('h2', string='Graduate Program Fees').find_next('ul')
li_tags = ul.find_all('li')

data_2 = []
for li in li_tags:
    item = li.get_text()
    description, cost = [x.strip() for x in item.split(':')]
    data_2.append([description, cost])

df2 = pd.DataFrame(data_2, columns=["Description", "Cost"])

In [6]:
tbl = soup.table

In [7]:
df3 = pd.read_html(str(tbl))[0]

/var/folders/jl/k_qw56hn2dx0cbwjpqwf8k3m0000gn/T/ipykernel_48140/1541087482.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3 = pd.read_html(str(tbl))[0]


In [8]:
merged_df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
merged_df.to_csv(TABLE + '.csv')

In [9]:
command = f"jupyter nbconvert --to script {TABLE}"
os.system(command)

[NbConvertApp] Converting notebook Tuition.ipynb to script
[NbConvertApp] Writing 1738 bytes to Tuition.py


0